In [1]:
import numpy as np
import pandas as pd
import re
import scipy
from scipy.stats import ttest_ind
from scipy.stats import pearsonr
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

### Importing the CSV file into a dataframe and getting an idea of it's content and size

In [2]:
articles_df = pd.read_csv('/Users/manueltanguma/Downloads/WELLCOME_APCspend2013_forThinkful_CSV2.csv', encoding='latin-1')

In [3]:
articles_df.shape

(2127, 5)

In [4]:
articles_df.head(5)

,PMID/PMCID,Publisher,Journal title,Article title,COST (Â£) charged to Wellcome (inc VAT when charged)
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,Â£0.00
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,Â£2381.04
2,23043264 PMC3506128,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",Â£642.56
3,23438330 PMC3646402,ACS,J Med Chem,Orvinols with mixed kappa/mu opioid receptor a...,Â£669.64
4,23438216 PMC3601604,ACS,J Org Chem,Regioselective opening of myo-inositol orthoes...,Â£685.88


In [5]:
articles_df.tail(5)

,PMID/PMCID,Publisher,Journal title,Article title,COST (Â£) charged to Wellcome (inc VAT when charged)
2122,2901593,Wolters Kluwer Health,Circulation Research,Mechanistic Links Between Na+ Channel (SCN5A) ...,Â£1334.15
2123,3748854,Wolters Kluwer Health,AIDS,Evaluation of an empiric risk screening score ...,Â£1834.77
2124,3785148,Wolters Kluwer Health,Pediatr Infect Dis J,Topical umbilical cord care for prevention of ...,Â£1834.77
2125,PMCID:\n PMC3647051,Wolters Kluwer N.V./Lippinott,AIDS,Grassroots Community Organisations' Contributi...,Â£2374.52
2126,PMID: 23846567 (Epub July 2013),Wolters Kluwers,Journal of Acquired Immune Deficiency Syndromes,A novel community health worker tool outperfor...,Â£2034.75


In [6]:
articles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2127 entries, 0 to 2126
Data columns (total 5 columns):
PMID/PMCID                                              1928 non-null object
Publisher                                               2127 non-null object
Journal title                                           2126 non-null object
Article title                                           2127 non-null object
COST (Â£) charged to Wellcome (inc VAT when charged)    2127 non-null object
dtypes: object(5)
memory usage: 83.2+ KB


### Determine the five most common journals and the total articles for each. Next, calculate the mean, median, and standard deviation of the open-access cost per article for each journal . You will need to do considerable data cleaning in order to extract accurate estimates, and may want to look into data encoding methods if you get stuck. For a real bonus round, identify the open access prices paid by subject area.

In [7]:
articles_df.head(10)

,PMID/PMCID,Publisher,Journal title,Article title,COST (Â£) charged to Wellcome (inc VAT when charged)
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,Â£0.00
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,Â£2381.04
2,23043264 PMC3506128,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",Â£642.56
3,23438330 PMC3646402,ACS,J Med Chem,Orvinols with mixed kappa/mu opioid receptor a...,Â£669.64
4,23438216 PMC3601604,ACS,J Org Chem,Regioselective opening of myo-inositol orthoes...,Â£685.88
5,PMC3579457,ACS,Journal of Medicinal Chemistry,Comparative Structural and Functional Studies ...,Â£2392.20
6,PMC3709265,ACS,Journal of Proteome Research,Mapping Proteolytic Processing in the Secretom...,Â£2367.95
7,23057412 PMC3495574,ACS,Mol Pharm,Quantitative silencing of EGFP reporter gene b...,Â£649.33
8,PMCID: PMC3780468,ACS (Amercian Chemical Society) Publications,ACS Chemical Biology,A Novel Allosteric Inhibitor of the Uridine Di...,Â£1294.59
9,PMCID: PMC3621575,ACS (Amercian Chemical Society) Publications,ACS Chemical Biology,Chemical proteomic analysis reveals the drugab...,Â£1294.78


### Determining the 5 most common journals and the total articles for each.  Converting the journal title's to lowercase and then will use the groupby function.  Ultimately mean, median, and std will be determined.

In [8]:
articles_df['Journal title'] = articles_df['Journal title'].apply(lambda x: str(x).lower())

In [9]:
temp = articles_df['Journal title'].value_counts()[:5]

In [10]:
type(temp)

pandas.core.series.Series

In [11]:
topfivelist = list(temp.index)

In [12]:
topfivelist

['plos one',
 'journal of biological chemistry',
 'neuroimage',
 'plos pathogens',
 'plos genetics']

In [13]:
topfive_df = articles_df[articles_df['Journal title'].isin(topfivelist)]

In [14]:
topfive_df['Journal title'].value_counts()

plos one                           190
journal of biological chemistry     53
neuroimage                          29
plos genetics                       24
plos pathogens                      24
Name: Journal title, dtype: int64

In [15]:
topfive_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 320 entries, 16 to 1604
Data columns (total 5 columns):
PMID/PMCID                                              315 non-null object
Publisher                                               320 non-null object
Journal title                                           320 non-null object
Article title                                           320 non-null object
COST (Â£) charged to Wellcome (inc VAT when charged)    320 non-null object
dtypes: object(5)
memory usage: 15.0+ KB


In [16]:
templist = list(topfive_df.columns)

In [17]:
templist

['PMID/PMCID',
 'Publisher',
 'Journal title',
 'Article title',
 'COST (Â£) charged to Wellcome (inc VAT when charged)']

In [18]:
templist[4]='Cost'

In [19]:
topfive_df.columns = templist

In [20]:
topfive_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 320 entries, 16 to 1604
Data columns (total 5 columns):
PMID/PMCID       315 non-null object
Publisher        320 non-null object
Journal title    320 non-null object
Article title    320 non-null object
Cost             320 non-null object
dtypes: object(5)
memory usage: 15.0+ KB


In [21]:
topfive_df.head(5)

,PMID/PMCID,Publisher,Journal title,Article title,Cost
16,22610094,AMBSB,journal of biological chemistry,Annexin-1 interaction with FPR2/ALX,Â£265.67
60,PMC3576085,American Soc for Biochemistry and Molecular Bi...,journal of biological chemistry,Understanding how noncatalytic carbohydrate bi...,Â£1100.00
61,23239883 PMC3561570,American Society for Biochemistry and Molecula...,journal of biological chemistry,Molecular architecture and functional analysis...,Â£2259.64
64,PMCID: PMC3642348,American Society for Biochemistry and Molecula...,journal of biological chemistry,Human and viral golgi anti-apoptotic protein (...,Â£1019.71
65,PMID: 22992744 PMC3493908,American Society for Biochemistry and Molecula...,journal of biological chemistry,Dynamic exchange of myosin VI on endocytic str...,Â£1119.61


In [22]:
topfive_df['Cost'] = topfive_df['Cost'].str.split('£', expand=True)[1]

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [23]:
topfive_df.head(5)

,PMID/PMCID,Publisher,Journal title,Article title,Cost
16,22610094,AMBSB,journal of biological chemistry,Annexin-1 interaction with FPR2/ALX,265.67
60,PMC3576085,American Soc for Biochemistry and Molecular Bi...,journal of biological chemistry,Understanding how noncatalytic carbohydrate bi...,1100.00
61,23239883 PMC3561570,American Society for Biochemistry and Molecula...,journal of biological chemistry,Molecular architecture and functional analysis...,2259.64
64,PMCID: PMC3642348,American Society for Biochemistry and Molecula...,journal of biological chemistry,Human and viral golgi anti-apoptotic protein (...,1019.71
65,PMID: 22992744 PMC3493908,American Society for Biochemistry and Molecula...,journal of biological chemistry,Dynamic exchange of myosin VI on endocytic str...,1119.61


In [24]:
topfive_df.groupby('Journal title', sort=True)

In [25]:
topfive_df['Cost'] = pd.to_numeric(topfive_df['Cost'])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [26]:
topfive_df.groupby(['Journal title'])['Cost'].mean()

Journal title
journal of biological chemistry     20264.633962
neuroimage                           2215.168276
plos genetics                       84839.435000
plos one                            38751.291368
plos pathogens                     112525.162778
Name: Cost, dtype: float64

In [28]:
topfive_df.groupby(['Journal title'])['Cost'].median()

Journal title
journal of biological chemistry    1314.530
neuroimage                         2326.430
plos genetics                      1718.390
plos one                            898.475
plos pathogens                     1684.000
Name: Cost, dtype: float64

In [29]:
topfive_df.groupby(['Journal title'])['Cost'].std()

Journal title
journal of biological chemistry    137165.488398
neuroimage                            266.653947
plos genetics                      281865.707794
plos one                           189009.511974
plos pathogens                     322866.078569
Name: Cost, dtype: float64